# Installing the Libraries 

In [5]:
# !pip install langchain-huggingface
# !pip install langchain
# !pip install langchain-community langchain-core
# !pip install langchain-cohere
# !pip install -U langchain-community faiss-cpu langchain-openai tiktoken
# !pip install sentence-transformers

# Importing the Necessary Libraries

In [33]:
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_cohere import ChatCohere
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
import re
import string
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import nltk.translate.bleu_score as bleu
from collections import Counter
import re

# Configure the Environment

In [7]:
with open("/kaggle/input/key-111/langchain_open_api_key.txt",'r') as m:
    langchain_key=m.read()
with open("/kaggle/input/key-111/cohere_open_api_key.txt","r") as z:
    cohere_api_key=z.read()
os.environ['LANGCHAIN_API_KEY']=langchain_key
os.environ['COHERE_API_KEY']=cohere_api_key
with open("/kaggle/input/hugging-face-token/hugging_face_token.txt",'r') as k:
    hugging_face_token=k.read()
os.environ['HUGGINGFACE_API_TOKEN']=hugging_face_token

# Extracring the text from pdf

In [8]:
from pypdf import PdfReader 
reader=PdfReader("/kaggle/input/attention-all-you-need/1706.03762v7.pdf")

text = ""
for i in range(len(reader.pages)):
    page = reader.pages[i]
    text_of_page = page.extract_text()
    if text_of_page:
        text += text_of_page

# Embedding model and Cohere llm

In [9]:
llm = ChatCohere()
hf_embeddings = HuggingFaceEmbeddings()

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-16 11:00:31.555437: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 11:00:31.555545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 11:00:31.681014: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Vector-store(Faiss)

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                chunk_overlap=200,
                                                length_function=len,
                                                is_separator_regex=False,)
texts=text_splitter.create_documents([text])
db = FAISS.from_documents(texts, hf_embeddings)
retriever=db.as_retriever()

In [39]:
prompt_template= """You are a bot. Your job is to provide the answer with respect to the question asked by the user.
    You have to provide the answer based on the context provided by the user.If you dont know the answer then reply 
    sorry i dont know the answer,Don't provide the unnecessary answers Provide the answer in the best neat and clean way:
    context: {context}
    """

prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_template),
    ("user", "{input}")
])
chain = prompt | llm 

In [12]:
question=input("Enter your question\n")
context=retriever.invoke(question)
answer=chain.invoke({"input": question,"context":context})
print("The answer\n",answer.content)

enter your question
 explain how the transformers works?


The answer
 The Transformer is a machine learning model architecture that has revolutionized the field of natural language processing (NLP). It was introduced in a seminal paper titled "Attention is All You Need" by Vaswani et al. in 2017. The key innovation of the Transformer is its use of self-attention mechanisms and fully connected layers to process sequential data, such as text, in parallel.

Here's a high-level explanation of how the Transformer works:
1. Input Embeddings: The input to the Transformer model is typically a sequence of tokens, such as words or subwords. These tokens are first passed through an embedding layer, which converts them into dense vector representations.
2. Encoder:
   - The encoder is responsible for transforming the input sequence into a new representation that captures the contextual relationships between the tokens.
   - It is composed of a stack of identical layers, each containing two sub-layers: a multi-head self-attention mechanism and a simple, p

In [40]:
#generaed answer
gen_answers=answer.content
#reference answer
ref_answer=""
for i in range(len(context)):
    content=context[i].page_content
    ref_answer+=content

# **# Evalution of the LLM Generated Text**

# Calculate ROUGE score 

In [38]:
 def calculate_ROUGE(generated_summary, reference_summary, n):
    # Tokenize the generated summary and reference summary into n-grams
    generated_ngrams = generate_ngrams(generated_summary, n)
    reference_ngrams = generate_ngrams(reference_summary, n)
 
    # Calculate the recall score
    matching_ngrams = len(set(generated_ngrams) & set(reference_ngrams))
    recall_score = matching_ngrams / len(reference_ngrams)
 
    return recall_score
 
 
def generate_ngrams(text, n):
    # Preprocess text by removing punctuation and converting to lowercase
    text = re.sub(r'[^\w\s]', '', text.lower())
 
    # Generate n-grams from the preprocessed text
    words = text.split()
    ngrams = [tuple(words[i:i+n]) for i in range(len(words) - n + 1)]
 
    return ngrams
 
 

generated_summary = gen_answers
reference_summary = ref_answer
n = 1  
rouge_score = calculate_ROUGE(generated_summary, reference_summary, n)
print(f"ROUGE-{n} score: {rouge_score}")


ROUGE-1 score: 0.2397003745318352


# Calculate the BLEU score

In [37]:
def calculate_BLEU(generated_summary, reference_summary, n):
    # Tokenize the generated summary and reference summary
    generated_tokens = generated_summary.split()
    reference_tokens = reference_summary.split()
 
    # Calculate the BLEU score
    weights = [1.0 / n] * n  # Weights for n-gram precision calculation
    bleu_score = bleu.sentence_bleu([reference_tokens], generated_tokens, weights=weights)
 
    return bleu_score
 
 

generated_summary = gen_answers
reference_summary = ref_answer
n = 1
 
bleu_score = calculate_BLEU(generated_summary, reference_summary, n)
print(f"BLEU-{n} score: {bleu_score}")

BLEU-1 score: 0.2777777777777778
